In [36]:
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [37]:
## Load trained model, scaler, and encoders

model=tf.keras.models.load_model('model.h5')

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

with open('onehot_encoder_func.pkl', 'rb') as file:
    onehot_encoder_func = pickle.load(file)

with open('onehot_encoder_ST.pkl', 'rb') as file:
    onehot_encoder_ST = pickle.load(file)

with open('onehot_encoder_TN.pkl', 'rb') as file:
    onehot_encoder_TN = pickle.load(file)


In [38]:

## Example test data (multiple rows)

test_data = {
    'Functionality': 'Payment Gateway',
    'Story_Type': 'API Gateway Changes',
    'Core_Function': 'Yes',
    'Affects_Workflow': 'Yes',
    'Has_Workaround': 'No',
    'Severity_Score': 8,
    'Tester_Name': 'Alice',
    'Time_Taken_Minutes': 35
}


In [39]:
## One-hot encode 

func_encoded = onehot_encoder_func.transform([[test_data['Functionality']]]).toarray()
func_encoded_df = pd.DataFrame(func_encoded, columns=onehot_encoder_func.get_feature_names_out(['Functionality']))
func_encoded_df 


,Functionality_Checkout,Functionality_Login,Functionality_Payment Gateway,Functionality_Profile Management,Functionality_Search
0,0.0,0.0,1.0,0.0,0.0


In [40]:
ST_encoded = onehot_encoder_ST.transform([[test_data['Story_Type']]]).toarray()
ST_encoded_df = pd.DataFrame(ST_encoded, columns=onehot_encoder_ST.get_feature_names_out(['Story_Type']))
ST_encoded_df

,Story_Type_AEM Story,Story_Type_API Gateway Changes,Story_Type_Database Update,Story_Type_React Story,Story_Type_UI Enhancement
0,0.0,1.0,0.0,0.0,0.0


In [41]:
TN_encoded = onehot_encoder_TN.transform([[test_data['Tester_Name']]]).toarray()
TN_encoded_df = pd.DataFrame(TN_encoded, columns=onehot_encoder_TN.get_feature_names_out(['Tester_Name']))
TN_encoded_df

,Tester_Name_Alice,Tester_Name_Bob,Tester_Name_Charlie
0,1.0,0.0,0.0


In [42]:
input_df=pd.DataFrame([test_data])
input_df

,Functionality,Story_Type,Core_Function,Affects_Workflow,Has_Workaround,Severity_Score,Tester_Name,Time_Taken_Minutes
0,Payment Gateway,API Gateway Changes,Yes,Yes,No,8,Alice,35


In [43]:
## Encode categorical variables

categorical_cols = ['Core_Function', 'Affects_Workflow', 'Has_Workaround']
for col in categorical_cols:
    input_df[col] = label_encoder.transform(input_df[col])

In [44]:
input_df=pd.concat([input_df.drop("Functionality",axis=1),func_encoded_df],axis=1)

In [45]:
input_df=pd.concat([input_df.drop("Story_Type",axis=1),ST_encoded_df],axis=1)

In [46]:
input_df=pd.concat([input_df.drop("Tester_Name",axis=1),TN_encoded_df],axis=1)

In [47]:
input_df

,Core_Function,Affects_Workflow,Has_Workaround,Severity_Score,Time_Taken_Minutes,Functionality_Checkout,Functionality_Login,Functionality_Payment Gateway,Functionality_Profile Management,Functionality_Search,Story_Type_AEM Story,Story_Type_API Gateway Changes,Story_Type_Database Update,Story_Type_React Story,Story_Type_UI Enhancement,Tester_Name_Alice,Tester_Name_Bob,Tester_Name_Charlie
0,1,1,0,8,35,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [48]:
## Scaling the input data

input_scaled=scaler.transform(input_df)
input_scaled

array([[ 1.13389342,  1.79179416, -1.88982237,  2.62625739,  0.84253382,
        -0.4404882 , -0.51946248,  2.12538029, -0.58697344, -0.48038446,
        -0.60623541,  2.081666  , -0.42008403, -0.48038446, -0.50974751,
         1.38169856, -0.65465367, -0.74390729]])

In [ ]:
## Predict

prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 88ms/step


array([[0.4665173 , 0.03436855, 0.00529977, 0.49381435]], dtype=float32)

In [ ]:
predicted_class = np.argmax(prediction)  # index of highest prob
class_names = ["No Issues", "P1", "P2", "P3"]  # your label order

print(f"Predicted class: {class_names[predicted_class]}")
print(f"Probability: {prediction[0][predicted_class]:.2f}")

Predicted class: P1
Probability: 0.49
